In [56]:
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

from sklearn.compose import make_column_transformer, make_column_selector
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder

from sklearn.linear_model import LinearRegression

import numpy as np

In [57]:
fuel = pd.read_csv('fuel.csv')
fuel.drop(columns={'Unnamed: 0'}, inplace=True)

In [58]:
fuel.isna().sum()

EngDispl               0
NumCyl                 0
Transmission           0
FE                     0
AirAspirationMethod    0
NumGears               0
TransLockup            0
TransCreeperGear       0
DriveDesc              0
IntakeValvePerCyl      0
ExhaustValvesPerCyl    0
CarlineClassDesc       0
VarValveTiming         0
VarValveLift           0
dtype: int64

In [59]:
fuel.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1107 entries, 0 to 1106
Data columns (total 14 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   EngDispl             1107 non-null   float64
 1   NumCyl               1107 non-null   int64  
 2   Transmission         1107 non-null   object 
 3   FE                   1107 non-null   float64
 4   AirAspirationMethod  1107 non-null   object 
 5   NumGears             1107 non-null   int64  
 6   TransLockup          1107 non-null   int64  
 7   TransCreeperGear     1107 non-null   int64  
 8   DriveDesc            1107 non-null   object 
 9   IntakeValvePerCyl    1107 non-null   int64  
 10  ExhaustValvesPerCyl  1107 non-null   int64  
 11  CarlineClassDesc     1107 non-null   object 
 12  VarValveTiming       1107 non-null   int64  
 13  VarValveLift         1107 non-null   int64  
dtypes: float64(2), int64(8), object(4)
memory usage: 121.2+ KB


In [60]:
preprocessing = make_column_transformer(
                (StandardScaler(),make_column_selector(dtype_include=np.number)),
                (LabelEncoder(), make_column_selector(dtype_include=object)),
)

In [61]:
X = fuel.copy()
y = X.pop('FE')

X_train, X_valid, y_train, y_valid = train_test_split(X,y,test_size=0.2,random_state=121,shuffle=False)

X_train = X_train.dropna(axis=1,how='all')
X_valid = X_valid.dropna(axis=1, how='all')

X_train_pre = preprocessing.fit_transform(X_train)
X_valid_pre = preprocessing.fit_transform(X_valid)

y_train = np.log(y_train)
y_valid = np.log(y_valid)


#X = pd.DataFrame(preprocessing.fit_transform(X))

#y = np.log(y)


#df_train = fuel.sample(frac=0.7, random_state=121)
#df_valid = fuel.drop(df_train.index)

TypeError: fit_transform() takes 2 positional arguments but 3 were given

In [55]:
print(X_train_pre.shape,X_valid_pre.shape)

(885, 49) (222, 31)


In [42]:
X_valid_pre.shape

(222, 31)

In [31]:
X_valid.shape

(222, 31)

In [32]:
X_train

<885x49 sparse matrix of type '<class 'numpy.float64'>'
	with 11505 stored elements in Compressed Sparse Row format>

In [ ]:
y = fuel['FE']
X = fuel[['EngDispl', 'NumCyl', 'Transmission', 'AirAspirationMethod','NumGears', 'TransLockup', 
          'TransCreeperGear', 'DriveDesc','IntakeValvePerCyl', 'ExhaustValvesPerCyl', 'CarlineClassDesc',
          'VarValveTiming', 'VarValveLift']]

In [57]:
X = preprocessing.fit_transform(X)
y = np.log(y)

X_train, X_valid , y_train, y_valid = train_test_split(X,y,test_size=0.2,random_state=420)

In [62]:
input_shape = [X_train.shape[1]]

In [66]:
model = keras.Sequential([
        # First layers
        layers.Dense(units=32, activation='relu', input_shape=input_shape),
        # Second Layers
        layers.Dense(units=32,activation='relu'),
        # Final layers
        layers.Dense(units=1),
]
)

In [71]:
model.compile(optimizer='adam',loss='mae',)

In [74]:
history = model.fit(
            X_train, y_train,
            validation_data=(X_valid,y_valid),
            epochs=10,
            batch_size=50,
)

Epoch 1/10


InvalidArgumentError:  TypeError: 'SparseTensor' object is not subscriptable
Traceback (most recent call last):

  File "/Users/bt/opt/anaconda3/lib/python3.8/site-packages/tensorflow/python/ops/script_ops.py", line 242, in __call__
    return func(device, token, args)

  File "/Users/bt/opt/anaconda3/lib/python3.8/site-packages/tensorflow/python/ops/script_ops.py", line 131, in __call__
    ret = self._func(*args)

  File "/Users/bt/opt/anaconda3/lib/python3.8/site-packages/tensorflow/python/autograph/impl/api.py", line 302, in wrapper
    return func(*args, **kwargs)

  File "/Users/bt/opt/anaconda3/lib/python3.8/site-packages/tensorflow/python/keras/engine/data_adapter.py", line 513, in py_method
    return [slice_array(inp) for inp in flat_inputs]

  File "/Users/bt/opt/anaconda3/lib/python3.8/site-packages/tensorflow/python/keras/engine/data_adapter.py", line 513, in <listcomp>
    return [slice_array(inp) for inp in flat_inputs]

  File "/Users/bt/opt/anaconda3/lib/python3.8/site